### Data Cleaning

In [ ]:
# import important packages
import pandas as pd
import numpy as np

In [ ]:
#Loading data
la_crashes = pd.read_csv("la_car_crashes.csv")
#la_crashes.info()
#la_crashes.head()

In [ ]:
# Change date and time to datetime object
la_crashes["Date Occurred"] = pd.to_datetime(la_crashes['Date Occurred'])
la_crashes["Date Reported"] = pd.to_datetime(la_crashes["Date Reported"])
la_crashes["Time Occurred"] = la_crashes['Time Occurred'].astype(str).map(lambda n: n.zfill(4))
la_crashes["Time Occurred"] = pd.to_datetime(la_crashes['Time Occurred'], format = "%H%M")

In [ ]:
#print(la_crashes["Date Occurred"].max())
#print(la_crashes["Date Occurred"].min())

In [ ]:
#Filter out crashes that are after 1 July 2019
la_crashes = la_crashes[(la_crashes["Date Occurred"] < pd.to_datetime("2019-07-01", format = "%Y-%m-%d"))]

In [ ]:
#Filter out crashes that are after 8am and before 12pm since those crashes are unlikely to be related to NBA games
la_crashes = la_crashes[(la_crashes['Time Occurred'] <= pd.to_datetime("08:00", format = "%H:%M")) | 
                        (la_crashes['Time Occurred'] >= pd.to_datetime("12:00", format = "%H:%M"))]

In [ ]:
# Find the possible game date associated with each crash
la_crashes["date_associated"] = np.where((la_crashes['Time Occurred'] <= pd.to_datetime("08:00", format = "%H:%M")),
                                         (la_crashes['Date Occurred']- pd.Timedelta(days=1)),
                                         la_crashes['Date Occurred'])

In [ ]:
#la_crashes.info()
#la_crashes.tail()

In [ ]:
# Load NBA game and team data
nba_games = pd.read_csv("nba_games.csv")
#nba_games.info()
#nba_games.head()

nba_teams = pd.read_csv("nba_teams.csv")
#nba_teams.info()

In [ ]:
nba_teams = nba_teams[["NICKNAME", "TEAM_ID"]]
nba_games = nba_games.drop(["TEAM_ID_home"], axis = 1)
#Get the home team name
nba_games = pd.merge(nba_games, nba_teams, left_on="HOME_TEAM_ID", right_on="TEAM_ID").rename(columns={"NICKNAME": "HOME_NICKNAME"})
#Get the visitor team name
nba_games = pd.merge(nba_games, nba_teams, left_on="VISITOR_TEAM_ID", right_on="TEAM_ID").rename(columns={"NICKNAME": "VISITOR_NICKNAME"})

In [ ]:
#Convert game date to datetime object
nba_games = nba_games[["GAME_DATE_EST", "HOME_NICKNAME", "VISITOR_NICKNAME", "HOME_TEAM_WINS"]]
nba_games["GAME_DATE_EST"] = pd.to_datetime(nba_games["GAME_DATE_EST"])

In [ ]:
#print(nba_games["GAME_DATE_EST"].min())
#print(nba_games["GAME_DATE_EST"].max())

In [ ]:
#nba_games.info()
#nba_games.head()

In [ ]:
# find games involving Lakers or Clippers
la_nba_games = nba_games[(nba_games["HOME_NICKNAME"] == "Lakers") | 
                         (nba_games["HOME_NICKNAME"] == "Clippers") | 
                         (nba_games["VISITOR_NICKNAME"] == "Clippers") | 
                         (nba_games["VISITOR_NICKNAME"] == "Lakers")]

In [ ]:
#la_nba_games.info()
#la_nba_games.head()

In [ ]:
# Update the date_associated var and create is_game_day var to indicate whether there is a game on that day
la_game_days = la_nba_games["GAME_DATE_EST"].tolist()
la_crashes['is_la_game_day'] = la_crashes['date_associated'].isin(la_game_days)
la_crashes["date_associated"] = np.where((la_crashes['Time Occurred'] <= pd.to_datetime("08:00", format = "%H:%M")) & 
                                         (la_crashes['is_la_game_day'] == False),
                                         (la_crashes['date_associated'] + pd.Timedelta(days=1)),
                                         la_crashes['date_associated'])
la_crashes = la_crashes[la_crashes['date_associated'] <= nba_games["GAME_DATE_EST"].max()]
la_crashes['is_la_game_day'] = la_crashes['date_associated'].isin(la_game_days)

In [ ]:
#la_crashes.info()
#la_crashes.tail()

In [ ]:
#print(la_crashes['date_associated'].min())
#print(la_crashes['date_associated'].max())

In [ ]:
#Get game infos
la_crashes = pd.merge(la_crashes, la_nba_games, left_on="date_associated", right_on="GAME_DATE_EST", how = "left")
la_crashes = la_crashes.drop(["GAME_DATE_EST"], axis = 1)

In [ ]:
#la_crashes.info()
#la_crashes.head()

In [ ]:
#Aggregate by date and area
crashes_by_area = la_crashes.groupby(
    ['date_associated', 'Area Name']
).agg(
    {
        'Area ID': "count",
        'is_la_game_day': 'first',
        "HOME_NICKNAME": 'first',
        "VISITOR_NICKNAME": "first",
        "HOME_TEAM_WINS" : "first"
    }
).rename(columns={"Area ID": "num_crashes"}).reset_index()

In [ ]:
#crashes_by_area.info()
#print(crashes_by_area["date_associated"].min())
#print(crashes_by_area["date_associated"].max())

In [ ]:
#crashes_by_area.head()

In [ ]:
#Add year, month, day, day of week vars
crashes_by_area['day_of_week'] = crashes_by_area.apply(lambda d: d["date_associated"].weekday(), axis=1)
crashes_by_area['month'] = crashes_by_area.apply(lambda d: d["date_associated"].month, axis=1)
crashes_by_area['year'] = crashes_by_area.apply(lambda d: d["date_associated"].year, axis=1)
crashes_by_area['day'] = crashes_by_area.apply(lambda d: d["date_associated"].day, axis=1)

In [ ]:
#Add is_homegame var
#crashes_by_area['is_homegame'] = (crashes_by_area['HOME_NICKNAME'] == "Clippers") | (crashes_by_area['HOME_NICKNAME'] == "Lakers")

In [ ]:
#Create is_holiday var to determine if a date is a holiday
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()
holidays = cal.holidays(start=crashes_by_area["date_associated"].min(), end=crashes_by_area["date_associated"].max())

crashes_by_area['is_holiday'] = crashes_by_area["date_associated"].isin(holidays)

In [ ]:
#crashes_by_area.info()
#crashes_by_area.tail()

In [ ]:
# Load weather data
weather_data = pd.read_csv("weather_data.csv")
#weather_data.info()
#weather_data.head()

In [ ]:
#Drop redundant coloumns and convert date str to datetime object
weather_data = weather_data.drop(["year","month","Date"], axis = 1)
weather_data["date"] = pd.to_datetime(weather_data["date"])

In [ ]:
#print(weather_data["date"].min())
#print(weather_data["date"].max())

In [ ]:
# Merge weather info with the crashes dataset
crashes_by_area = pd.merge(crashes_by_area, weather_data, left_on="date_associated", right_on="date", how = "left").drop(["date"], axis = 1)

In [ ]:
#crashes_by_area.info()
#crashes_by_area.head()

In [ ]:
# Convert is_la_game_day, is_homegame, is_holiday, to dummy variables
crashes_by_area.is_la_game_day = crashes_by_area.is_la_game_day.replace({True: 1, False: 0})
#crashes_by_area.is_homegame = crashes_by_area.is_homegame.replace({True: 1, False: 0})
crashes_by_area.is_holiday = crashes_by_area.is_holiday.replace({True: 1, False: 0})

#crashes_by_area['is_la_game_day'] = pd.Categorical(crashes_by_area.is_la_game_day)
#crashes_by_area['is_homegame'] = pd.Categorical(crashes_by_area.is_homegame)
crashes_by_area['is_holiday'] = pd.Categorical(crashes_by_area.is_holiday)
crashes_by_area['HOME_TEAM_WINS'] = pd.Categorical(crashes_by_area.HOME_TEAM_WINS)
crashes_by_area = crashes_by_area.rename(columns = {"Area Name":"area_name"})
crashes_by_area['area_name'] = pd.Categorical(crashes_by_area.area_name)
crashes_by_area['day_of_week'] = pd.Categorical(crashes_by_area.day_of_week)
crashes_by_area['month'] = pd.Categorical(crashes_by_area.month)
crashes_by_area['year'] = pd.Categorical(crashes_by_area.year)
crashes_by_area['day'] = pd.Categorical(crashes_by_area.day)

In [ ]:
# create is_home_game_day dummy variable 
la_nba_home_games = la_nba_games[(la_nba_games["HOME_NICKNAME"] == "Lakers") | 
                         (la_nba_games["HOME_NICKNAME"] == "Clippers")]
la_nba_home_game_dates = la_nba_home_games["GAME_DATE_EST"].tolist()
crashes_by_area["is_home_game_day"] = crashes_by_area['date_associated'].isin(la_nba_home_game_dates)
crashes_by_area.is_home_game_day = crashes_by_area.is_home_game_day.replace({True: 1, False: 0})
crashes_by_area['is_home_game_day'] = pd.Categorical(crashes_by_area.is_home_game_day)


In [ ]:
#crashes_by_area.info()
#crashes_by_area.head()

In [ ]:
#print(crashes_by_area["date_associated"].min())
#print(crashes_by_area["date_associated"].max())

In [ ]:
#la_crashes[la_crashes["date_associated"] == pd.to_datetime("2009-12-31")].info()
#la_nba_games[la_nba_games["GAME_DATE_EST"] == pd.to_datetime("2010-01-02", format="%Y-%m-%d")].info()

In [ ]:
#la_crashes.info()

In [ ]:
#Repeat the same process for men and women
la_crashes_female = la_crashes[la_crashes["Victim Sex"] == "F"]
la_crashes_male = la_crashes[la_crashes["Victim Sex"] == "M"]

In [ ]:
crashes_by_area_female = la_crashes_female.groupby(
    ['date_associated', 'Area Name']
).agg(
    {
        'Area ID': "count",
        'is_la_game_day': 'first',
        "HOME_NICKNAME": 'first',
        "VISITOR_NICKNAME": "first",
        "HOME_TEAM_WINS" : "first"
    }
).rename(columns={"Area ID": "num_crashes"}).reset_index()

crashes_by_area_male = la_crashes_male.groupby(
    ['date_associated', 'Area Name']
).agg(
    {
        'Area ID': "count",
        'is_la_game_day': 'first',
        "HOME_NICKNAME": 'first',
        "VISITOR_NICKNAME": "first",
        "HOME_TEAM_WINS" : "first"
    }
).rename(columns={"Area ID": "num_crashes"}).reset_index()

In [ ]:
crashes_by_area_female['day_of_week'] = crashes_by_area_female.apply(lambda d: d["date_associated"].weekday(), axis=1)
crashes_by_area_female['month'] = crashes_by_area_female.apply(lambda d: d["date_associated"].month, axis=1)
crashes_by_area_female['year'] = crashes_by_area_female.apply(lambda d: d["date_associated"].year, axis=1)
crashes_by_area_female['day'] = crashes_by_area_female.apply(lambda d: d["date_associated"].day, axis=1)
crashes_by_area_male['day_of_week'] = crashes_by_area_male.apply(lambda d: d["date_associated"].weekday(), axis=1)
crashes_by_area_male['month'] = crashes_by_area_male.apply(lambda d: d["date_associated"].month, axis=1)
crashes_by_area_male['year'] = crashes_by_area_male.apply(lambda d: d["date_associated"].year, axis=1)
crashes_by_area_male['day'] = crashes_by_area_male.apply(lambda d: d["date_associated"].day, axis=1)

In [ ]:
crashes_by_area_female['is_holiday'] = crashes_by_area_female["date_associated"].isin(holidays)
crashes_by_area_male['is_holiday'] = crashes_by_area_male["date_associated"].isin(holidays)

In [ ]:
crashes_by_area_female = pd.merge(crashes_by_area_female, weather_data, left_on="date_associated", right_on="date", how = "left").drop(["date"], axis = 1)
crashes_by_area_male = pd.merge(crashes_by_area_male, weather_data, left_on="date_associated", right_on="date", how = "left").drop(["date"], axis = 1)

In [ ]:
crashes_by_area_female.is_la_game_day = crashes_by_area_female.is_la_game_day.replace({True: 1, False: 0})
#crashes_by_area_female.is_homegame = crashes_by_area_female.is_homegame.replace({True: 1, False: 0})
crashes_by_area_female.is_holiday = crashes_by_area_female.is_holiday.replace({True: 1, False: 0})
crashes_by_area_male.is_la_game_day = crashes_by_area_male.is_la_game_day.replace({True: 1, False: 0})
#crashes_by_area_male.is_homegame = crashes_by_area_male.is_homegame.replace({True: 1, False: 0})
crashes_by_area_male.is_holiday = crashes_by_area_male.is_holiday.replace({True: 1, False: 0})

In [ ]:
#crashes_by_area['is_homegame'] = pd.Categorical(crashes_by_area.is_homegame)
crashes_by_area_female['is_holiday'] = pd.Categorical(crashes_by_area_female.is_holiday)
crashes_by_area_female['HOME_TEAM_WINS'] = pd.Categorical(crashes_by_area_female.HOME_TEAM_WINS)
crashes_by_area_female = crashes_by_area_female.rename(columns = {"Area Name":"area_name"})
crashes_by_area_female['area_name'] = pd.Categorical(crashes_by_area_female.area_name)
crashes_by_area_female['day_of_week'] = pd.Categorical(crashes_by_area_female.day_of_week)
crashes_by_area_female['month'] = pd.Categorical(crashes_by_area_female.month)
crashes_by_area_female['year'] = pd.Categorical(crashes_by_area_female.year)
crashes_by_area_female['day'] = pd.Categorical(crashes_by_area_female.day)
#crashes_by_area['is_homegame'] = pd.Categorical(crashes_by_area.is_homegame)
crashes_by_area_male['is_holiday'] = pd.Categorical(crashes_by_area_male.is_holiday)
crashes_by_area_male['HOME_TEAM_WINS'] = pd.Categorical(crashes_by_area_male.HOME_TEAM_WINS)
crashes_by_area_male = crashes_by_area_male.rename(columns = {"Area Name":"area_name"})
crashes_by_area_male['area_name'] = pd.Categorical(crashes_by_area_male.area_name)
crashes_by_area_male['day_of_week'] = pd.Categorical(crashes_by_area_male.day_of_week)
crashes_by_area_male['month'] = pd.Categorical(crashes_by_area_male.month)
crashes_by_area_male['year'] = pd.Categorical(crashes_by_area_male.year)
crashes_by_area_male['day'] = pd.Categorical(crashes_by_area_male.day)

In [ ]:
crashes_by_area_male["is_home_game_day"] = crashes_by_area_male['date_associated'].isin(la_nba_home_game_dates)
crashes_by_area_male.is_home_game_day = crashes_by_area_male.is_home_game_day.replace({True: 1, False: 0})
crashes_by_area_male['is_home_game_day'] = pd.Categorical(crashes_by_area_male.is_home_game_day)

crashes_by_area_female["is_home_game_day"] = crashes_by_area_female['date_associated'].isin(la_nba_home_game_dates)
crashes_by_area_female.is_home_game_day = crashes_by_area_female.is_home_game_day.replace({True: 1, False: 0})
crashes_by_area_female['is_home_game_day'] = pd.Categorical(crashes_by_area_female.is_home_game_day)